# MIP

In [1]:
from pulp import *

# Read the file and initialize the variables
with open("../data/inst03.dat", "r") as file:
    lines = file.readlines()

# Create a new model
model = LpProblem("CourierRoutingProblem", LpMinimize)

In [2]:
## Parse the values from the file
m = int(lines[0].strip())  # Number of couriers
n = int(lines[1].strip())  # Number of items
l_values = list(map(int, lines[2].split()))  # Maximum load for each courier
s_values = list(map(int, lines[3].split()))  # Size of each item
D_values = [list(map(int, line.split())) for line in lines[4:]]  # Distance matrix
#l = LpVariable.dicts("l", range(m), lowBound=0, cat="Integer")  # Maximum load decision variables
#for i in range(m):
#    l[i].bounds = (l_values[i], l_values[i])
#    model.addConstraint(l[i] == l_values[i])
#
#
#s = LpVariable.dicts("s", range(n), lowBound=0, cat="Integer")  # Size of each item decision variables
#for i in range(n):
#    s[i].bounds = (s_values[i], s_values[i])
#    model.addConstraint(s[i] == s_values[i])
#
#
#D = LpVariable.dicts("D", (range(n + 1), range(n + 1)), lowBound=0, cat="Integer")  # Distance matrix decision variables
#
#for i in range(n + 1):
#    for j in range(n + 1):
#        D[i][j].bounds = (D_values[i][j], D_values[i][j])
#        #D[i][j].setInitialValue(D_values[i][j])
#        model.addConstraint(D[i][j] == D_values[i][j])
#
#routes = LpVariable.dicts("routes", (range(n+1), range(n + 1)), lowBound=0, upBound=m, cat="Integer")  # Routes for each courier OLD MODEL
routes = [LpVariable.dicts(f"routes_{i+1}", (range(n+1), range(n + 1)), lowBound=0, upBound=1, cat="Integer") for i in range(m)]


In [3]:
# objective function 
objective = [lpSum([D_values[t][j] * routes[i][t][j] for t in range(n+1) for j in range(n+1)]) for i in range(m)]


### CONSTRAINTS

In [4]:
##Domain constraint
model += lpSum(routes) == (n+m)

#No 1 values on diagonals
model += lpSum([routes[d][i][i] for d in range(m) for i in range(n+1)]) == 0

#At most 1 value in the same columns for each courier except last column
for j in range(n):
    model += lpSum([routes[d][i][j] for d in range(m) for i in range(n+1)]) <= 1

#At most 1 value in the same row for each courier except last row
for i in range(n):
    model += lpSum([routes[d][i][j] for d in range(m) for j in range(n+1)]) <= 1


for d in range(m):

    #At least 1 value for each courier
    model += lpSum(routes[d]) >= 2

    #Exactly 1 value in the last row for each courier   
    model += lpSum([routes[d][n][j]  for j in range(n+1)])== 1

    #Exactly 1 value in the last column for each courier
    model += lpSum([routes[d][i][n]  for i in range(n+1)]) == 1

    #Loads constraint
    model += lpSum([lpSum(routes[d][i]) * s_values[i] for i in range(n)]) <= l_values[d]
    
    for i in range(n+1):
        #The path must be coherent => there should exist a coherent path from the first destination point to the origin
        #It's implemented by verifying that the sum of the values on the row 1 is equal to the sum of values in the comumn 1 => 
        #if there's a row in the ith row there should be a 1 also in the ith column and viceversa

        model += (lpSum([routes[d][i][t] for t in range(n+1) ]) == lpSum([routes[d][t][i] for t in range(n+1)]))

        for j in range(n):
            ##Couriers cannot go back to a destination point already visited, except if it is the origin point
            #Example   o -> 1   1 -> is allowed
            model += (routes[d][i][j] + routes[d][j][i]) <=1


    
#TODO Symmetry breaking contraints?




In [5]:
maximum = LpVariable("maximum", lowBound=0, cat="Integer")

for i in range(len(objective)):
    model.addConstraint(constraint = (maximum>=objective[i])) 

model.setObjective(maximum)

# Solve the model
model.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[model.status])

if model.status == LpStatusOptimal:
    length = model.objective.value()
   
else:
    print("No solution found.")

Status: Optimal


In [6]:
#Print results
routes_values = []
for d in range(m):
    r = [n+1] * n
    pos = 0
    index = n
    while True: 
        index = [routes[d][index][j].varValue for j in range(n+1)].index(1.0)
        r[pos] = index+1
        pos+=1
        if(index == n):
            routes_values.append(r)
            break
print("---ROUTES---")
print(routes_values)

print("---DISTANCES---")
print([sum([D_values[t][j] * routes[i][t][j].varValue for t in range(n+1) for j in range(n+1)]) for i in range(m)])
print("---ROUTES MASK ---")
for d in range(m):
    print(f"---COURIER {d+1}---")
    for i in range(n+1):
        print([routes[d][i][j].varValue for j in range(n + 1)])
        
print("---MAXIMUM---")
print(length)

---ROUTES---
[[3, 6, 5, 8, 8, 8, 8], [2, 4, 8, 8, 8, 8, 8], [1, 7, 8, 8, 8, 8, 8]]
---DISTANCES---
[12.0, 10.0, 12.0]
---ROUTES MASK ---
---COURIER 2---
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---COURIER 3---
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
---MAXIMUM---
12.0
